<a href="https://colab.research.google.com/github/ipeirotis/sql_autograding/blob/main/few_shot_trial_7_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai
!pip install -q google-cloud-secret-manager
!pip3 install -U -q PyMySQL sqlalchemy sql_magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 9.0 MB/s eta 0:00:00


In [2]:
from google.colab import auth

# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [3]:
from google.cloud import secretmanager


def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload of the given secret version and return it.

    Args:
        project_id (str): Google Cloud project ID.
        secret_id (str): ID of the secret to access.
        version_id (str): ID of the version to access.
    Returns:
        str: The secret version's payload, or None if
        the version does not exist.
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")


openai_key = access_secret_version("sql-autograding", "openai-gpt4-32k", "3")

In [4]:
import openai

openai.api_key = openai_key

In [5]:
from google.colab import files
import io
import pandas as pd

In [6]:
import gcsfs

fs = gcsfs.GCSFileSystem(project="sql_autograding")
with fs.open("gs://sql_autograding/cleaned_response.csv") as f:
    data = pd.read_csv(f)

In [7]:
data = data.drop(columns="Unnamed: 0")
data.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\n-1 pts for using t...,"SELECT O.OrderID, O.CustomerID\n&#160; &#160; ...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n..."
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN


## Table Schemas

In [8]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [9]:
db_list = [
    "flights",
    "imdb",
    "music",
    "restaurants",
    "facebook",
    "northwind",
    "collisions",
]


def db_schema(db_name):
    """
    Connects to a database and returns the schema of each table in the database.

    The function connects to a specific database using SQLAlchemy. It then retrieves the list of tables in
    the database and for each table, it queries the schema (i.e., the list of fields/columns) and stores it
    in a dictionary. The function returns a list of such dictionaries, with each dictionary representing a table
    and its corresponding schema.

    Args:
        db_name (str): The name of the database to connect to and retrieve schemas from.

    Returns:
        list: A list of dictionaries, with each dictionary containing the name of a table as the key and a
              list of its fields as the value.

    Example:
        >>> db_schema('flights')
        {'m_airports': ['airport', 'state', 'state_name']},
        {'m_ticket_prices': ['origin', 'dest', 'carrier', 'fare', 'fare_per_mile','passengers', 'distance']
        ...
        },
    """

    student_password = access_secret_version(
        "sql-autograding", "db_student_password", "1"
    )

    conn_string = (
        "mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4".format(
            host="db.ipeirotis.org",
            user="student",
            password=student_password,
            db=db_name,
            encoding="utf8mb4",
        )
    )
    engine = create_engine(conn_string)

    with engine.begin() as conn:
        tables = pd.read_sql_query(sql=text("show tables"), con=conn)
        tables = tables.iloc[:, -1].tolist()

        schema_list = []
        for t in tables:
            d = pd.read_sql_query(sql=text(f"describe {t}"), con=conn)
            table_schema = {t: d.loc[:, "Field"].tolist()}
            schema_list.append(table_schema)
    return schema_list

In [10]:
flights_schema = db_schema("flights")
imdb_schema = db_schema("imdb")
music_schema = db_schema("music")
restaurants_schema = db_schema("restaurants")
facebook_schema = db_schema("facebook")
northwind_schema = db_schema("northwind")
collisions_schema = db_schema("collisions")

print(facebook_schema)

[{'Concentration': ['ProfileID', 'Concentration']}, {'FavoriteBooks': ['ProfileID', 'Book']}, {'FavoriteMovies': ['ProfileID', 'Movie']}, {'FavoriteMusic': ['ProfileID', 'Music']}, {'FavoriteTVShows': ['ProfileID', 'TVShow']}, {'Hobbies': ['ProfileID', 'Hobby']}, {'LookingFor': ['ProfileID', 'LookingFor']}, {'Orientation': ['ProfileID', 'InterestedIn']}, {'Profiles': ['ProfileID', 'Name', 'MemberSince', 'LastUpdate', 'School', 'Status', 'Sex', 'Birthday', 'AIM', 'Website', 'PoliticalViews', 'Geography', 'HighSchool', 'HomeTown', 'HomeState', 'Residence', 'CurrentAddress', 'CurrentTown', 'CurrentState']}, {'Relationship': ['ProfileID', 'Status']}, {'chart': ['music', 'gender', 'cnt', 'music_rank']}, {'popular_music': ['music', 'cnt']}, {'popular_music_by_sex': ['music', 'gender', 'cnt']}]


In [11]:
schema_mapping = {
    "flights": flights_schema,
    "imdb": imdb_schema,
    "music": music_schema,
    "restaurants": restaurants_schema,
    "facebook": facebook_schema,
    "northwind": northwind_schema,
    "collisions": collisions_schema,
    "['northwind', 'flights']": (northwind_schema, flights_schema),
}

In [12]:
data["schema"] = data["Database"].map(schema_mapping)
data.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'..."
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\n-1 pts for using t...,"SELECT O.OrderID, O.CustomerID\n&#160; &#160; ...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'..."
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'..."
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."


In [13]:
from bs4 import BeautifulSoup

# Create a function to clean up a single text string
def clean_html_content(text):
    if pd.isna(text):
        return text
    if not isinstance(text, str):
        text = str(text)
    try:
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()
    except:
        return text


# Apply the function to every text column in your DataFrame
for col in data.columns:
    if col in ("Database", "schema"):
        continue
    if data[col].dtype == object:  # if the column is a text column
        data[col] = data[col].apply(clean_html_content)

<ipython-input-13-f8217fd0dcfa>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


## Group questions

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['QuestionId'], random_state=1234)

In [16]:
print(data['QuestionId'].sort_values().value_counts())

5,119,720    86
5,119,725    86
5,119,723    86
5,119,721    86
5,119,719    85
             ..
5,117,074     1
5,117,073     1
5,117,072     1
5,117,071     1
5,117,070     1
Name: QuestionId, Length: 64, dtype: int64


In [17]:
df = data[data["QuestionId"] == "5,119,721"]
df.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
158,"Databases for Business Analytics, Spring'23","251,685",Assignment 3: Joins,"295,367",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"select T.track_name, L.album_name, a.artist_na...",1.0,my2204,NaN,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
219,"Databases for Business Analytics, Spring'23","251,685",Assignment 3: Joins,"295,367",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"SELECT track_name, artist_name, album_name\nFR...",0.5,ql2084,NaN,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
289,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 3: Joins,"301,296",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"USE music;\nSHOW tables;\nSELECT track_name, a...",1.0,soj206,NaN,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
332,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 3: Joins,"301,296",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"SELECT T.track_name, A1.artist_name, A2.album_...",0.7,pp1711,\n\n\nAnswer Key\n\n\n\n\n\n\n\nSELECT track_n...,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,The feedback from the TA was \n\n\nAnswer Key\...,"[{'album': ['artist_id', 'album_id', 'album_na..."
420,"Databases for Business Analytics, Spring'23","251,685",Assignment 3: Joins,"295,367",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"Select T.track_name, L.album_name, A.artist_na...",1.0,sek495,NaN,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."


In [18]:
qids = df["QuestionId"].unique()
print(qids)

['5,119,721']


## Mega Prompt

===============================

a. We are working with the XXX database, which has the following tables:
artist(id, first_name, last_name)
album(id, artist_id, name)
track....

b. The request to the student is "Fetch all the tracks for user X"

c. The model answer is ..... (note that we may have multiple correct
answers, the model answer is just an example)

d. The student answer was .....

e. Previously, students have submitted these answers and got back
these responses and grades:

e1. submission: ..... , grade: ..... , feedback

e2. submission: ..... , grade: ..... , feedback

e3. submission: ..... , grade: ..... , feedback

Please provide a grade and feedback for the student

===============================

In [8]:
from typing import List


def generate_submissions_string(
    train_df: pd.DataFrame, submission_size: int = None
) -> str:
    """
    Generate a string that concatenates student submissions up to the given size.

    Args:
        train_df (pd.DataFrame): The DataFrame containing the training data.
        submission_size (int, optional): The number of submissions to include in the string. If None, include all submissions.

    Returns:
        str: A string containing the concatenated submissions.
    """
    submissions = ""
    for i, row in train_df.iterrows():
        if submission_size is not None and i >= submission_size:
            break
        submission = row["InputUserAnswer"]
        grade = row["Score"]
        full_grade = row["full_score"]
        feedback = row["feedback"]
        submissions += f"e{i}. submission: {submission}, grade: {grade}/{full_grade}, feedback: {feedback} \\n\\n"
    return submissions


def generate_mega_prompt_for_id(
    df: pd.DataFrame, id: str, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        id (str): The QuestionId to generate the mega prompt for.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = df[df["QuestionId"] == id]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return None  # TODO: Check what to return in this scenario
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]

      test = temp.loc[0, "InputUserAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade (out of 10 points) and feedback for the student
        """
      return mega_p

    # making test size 1
    train_df, test_df = train_test_split(temp, test_size=1, random_state=1234)
    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    submissions = generate_submissions_string(train_df, submission_size)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]

    test = test_df.loc[0, "InputUserAnswer"]

    mega_p = f"""
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

      Please provide a grade and feedback for the student
      """
    return mega_p


def generate_mega_prompts(
    df: pd.DataFrame, submission_size: int = None
) -> pd.DataFrame:
    """
    Generate a DataFrame of mega prompts for each unique QuestionId in the given DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        submission_size (int, optional): The number of submissions to include in each mega prompt. If None, include all submissions.

    Returns:
        pd.DataFrame: A DataFrame containing the mega prompts.
    """
    # mega_df = pd.DataFrame(columns=['QuestionId', 'MegaPrompt'])
    result = []
    qids = df["QuestionId"].unique()

    for id in qids:
        mega_p = generate_mega_prompt_for_id(df, id, submission_size)
        new_row = {"QuestionId": id, "MegaPrompt": mega_p}
        result.append(new_row)

    mega_df = pd.DataFrame(result)

    return mega_df

In [20]:
# for all questions each with all submissions => new df
mega_df_all = generate_mega_prompts(data)
mega_df_all.head()

,QuestionId,MegaPrompt
0,"4,969,729",\n a. We are working with the ['northwind...
1,"4,969,713",\n a. We are working with the ['northwind...
2,"4,969,711",\n a. We are working with the ['northwind...
3,"4,969,710",\n a. We are working with the ['northwind...
4,"4,969,709",\n a. We are working with the ['northwind...


In [21]:
mega_df_all.shape

(64, 2)

In [22]:
print(mega_df_all.loc[0, "MegaPrompt"])


      a. We are working with the ['northwind', 'flights'] database, which has the following tables: ([{'Categories': ['CategoryID', 'CategoryName', 'Description']}, {'CustomerGroupThresholds': ['CustomerGroupName', 'RangeBottom', 'RangeTop']}, {'Customers': ['CustomerID', 'CompanyName', 'ContactName', 'ContactTitle', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'Phone', 'Fax']}, {'Employees': ['EmployeeID', 'LastName', 'FirstName', 'Title', 'TitleOfCourtesy', 'BirthDate', 'HireDate', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'HomePhone', 'Extension', 'Notes', 'ReportsTo', 'PhotoPath']}, {'OrderDetails': ['OrderID', 'ProductID', 'UnitPrice', 'Quantity', 'Discount']}, {'Orders': ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'Freight', 'ShipName', 'ShipAddress', 'ShipCity', 'ShipRegion', 'ShipPostalCode', 'ShipCountry']}, {'Products': ['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'QuantityPerUnit', 'UnitPric

In [23]:
# select a specific question and control submission size
df = data[data["QuestionId"] == "5,168,443"]

df0 = generate_mega_prompts(df, 0)
df0.head()

,QuestionId,MegaPrompt
0,"5,168,443",\n a. We are working with the flights dat...


In [24]:
df5 = generate_mega_prompts(df, 5)
df10 = generate_mega_prompts(df, 10)
df20 = generate_mega_prompts(df, 20)
df_all = generate_mega_prompts(df, len(df))

## GPT4

In [9]:
def GPT3_generation(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [10]:
def GPT4_generation(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4-32k",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [27]:
print(GPT4_generation(df0.loc[0, "MegaPrompt"]))

The student's answer is correct. 

Grade: 10/10

Feedback: Great job! Your SQL query correctly calculates each airport's national and state rank in terms of departing passengers. You've correctly used the RANK() function and joined the necessary tables. Keep up the good work!


In [28]:
print(GPT4_generation(df5.loc[0, "MegaPrompt"]))

The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the passengers_per_airport and m_airports tables on the airport column. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. Your join operation is also correct. Keep up the good work!


In [29]:
print(GPT4_generation(df10.loc[0, "MegaPrompt"]))

The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the passengers_per_airport and m_airports tables on the airport column. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function and joined the tables correctly. Keep up the good work!


In [30]:
print(GPT4_generation(df20.loc[0, "MegaPrompt"]))

The student's answer is correct. They have correctly used the RANK() function to calculate the national and state ranks based on the total number of passengers. They have also correctly joined the m_airports table to get the state of each airport. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function to calculate the national and state ranks. Your join operation is also correct. Keep up the good work!


In [31]:
print(GPT4_generation(df_all.loc[0, "MegaPrompt"]))

The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the passengers_per_airport and m_airports tables on the 'origin' and 'airport' columns respectively. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function and joined the tables correctly. Keep up the good work!


In [32]:
def test_shots(qid, n):
    df = data[data["QuestionId"] == qid]
    df_n = generate_mega_prompts(df, n)
    # df_n.head()
    print(f"{n} submissions:")
    print(GPT4_generation(df_n.loc[0, "MegaPrompt"]))

In [33]:
for i in range(0, 21, 5):
    test_shots("5,168,443", i)

0 submissions:
The student's answer is correct. 

Grade: 10/10

Feedback: Great job! Your SQL query correctly calculates the national and state rank for each airport based on the number of departing passengers. You've correctly used the RANK() function and joined the necessary tables. Keep up the good work!
5 submissions:
The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the passengers_per_airport and m_airports tables on the airport column. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function and joined the tables correctly. Keep up the good work!
10 submissions:
The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the passengers_per_airpor

In [34]:
mega_df_all.shape
mega_df_all.head(20)

,QuestionId,MegaPrompt
0,"4,969,729",\n a. We are working with the ['northwind...
1,"4,969,713",\n a. We are working with the ['northwind...
2,"4,969,711",\n a. We are working with the ['northwind...
3,"4,969,710",\n a. We are working with the ['northwind...
4,"4,969,709",\n a. We are working with the ['northwind...
5,"4,969,708",\n a. We are working with the ['northwind...
6,"4,969,707",\n a. We are working with the ['northwind...
7,"4,969,706",\n a. We are working with the ['northwind...
8,"4,969,705",\n a. We are working with the ['northwind...
9,"5,119,646",\n a. We are working with the music datab...


In [35]:
from tqdm import tqdm

In [ ]:
for i, p in tqdm(mega_df_all.iterrows()):
    # print(i, len(p['MegaPrompt'].split()))
    p["feedback"] = GPT4_generation(p["MegaPrompt"])

mega_df_all.head()

In [ ]:
mega_df_all["feedback"] = mega_df_all["MegaPrompt"].apply(GPT4_generation)
mega_df_all.head()

## DataFrame with ZeroShotPrompt

In [37]:
df_new = data.copy()
df_new.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'..."
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\n-1 pts for using t...,"SELECT O.OrderID, O.CustomerID\n , SUM(D.Qu...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'..."
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'..."
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."


In [38]:
mega_df_0 = generate_mega_prompts((df_new.iloc[0]).to_frame().T,0)
mega_df_0.head()

,QuestionId,MegaPrompt
0,"4,969,729",\n a. We are working with the ['northwi...


In [39]:
print(mega_df_0.loc[0,'MegaPrompt'])


        a. We are working with the ['northwind', 'flights'] database, which has the following tables: ([{'Categories': ['CategoryID', 'CategoryName', 'Description']}, {'CustomerGroupThresholds': ['CustomerGroupName', 'RangeBottom', 'RangeTop']}, {'Customers': ['CustomerID', 'CompanyName', 'ContactName', 'ContactTitle', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'Phone', 'Fax']}, {'Employees': ['EmployeeID', 'LastName', 'FirstName', 'Title', 'TitleOfCourtesy', 'BirthDate', 'HireDate', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'HomePhone', 'Extension', 'Notes', 'ReportsTo', 'PhotoPath']}, {'OrderDetails': ['OrderID', 'ProductID', 'UnitPrice', 'Quantity', 'Discount']}, {'Orders': ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'Freight', 'ShipName', 'ShipAddress', 'ShipCity', 'ShipRegion', 'ShipPostalCode', 'ShipCountry']}, {'Products': ['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'QuantityPerUnit', 'UnitPr

In [40]:
print(GPT4_generation(mega_df_0.loc[0, "MegaPrompt"]))

Grade: 9/10

Feedback: Your answer is almost perfect. You have correctly used subqueries and the CASE WHEN structure to define the three groups. However, you have calculated the discount applied as a percentage, while the question asked for the total discount applied. You should have calculated the total discount as `SpendBeforeDiscount - SpendAfterDiscount`. Also, remember to use COALESCE function to handle NULL values. Keep up the good work!


In [41]:
print(GPT3_generation(mega_df_0.loc[0, "MegaPrompt"]))

Grade: 7/10

Feedback: 
- The student's query is mostly correct and achieves the desired result.
- The subquery to calculate the total amount spent before discounts is correct.
- The CASE WHEN structure is used correctly to define the three groups based on the total amount spent after discounts.
- However, there are a few minor issues with the query:
  - The alias "SpendAfterDiscount" is not defined in the subquery, causing an error. It should be "total_discount" instead.
  - The calculation for the discountApplied column is incorrect. It should be (total_discount / SpendBeforeDiscount) instead of (1 - (SpendafterDiscount / SpendBeforeDiscount)).
  - The JOIN condition between Orders and Customers tables is missing in the subquery. It should be "INNER JOIN Customers C ON O.CustomerId = C.CustomerID".
- Overall, the student's query demonstrates a good understanding of subqueries and CASE WHEN structure, but there are some minor errors that need to be corrected.


In [17]:
def parse_grade_feedback(text):
  colon_pos = text.find(":")
  divider_pos = text.find("/")

  grade = text[colon_pos+2:divider_pos]

  feedback_start = text.find("Feedback: ")
  if feedback_start != -1:
      feedback = text[feedback_start + len("Feedback: "):]
  else:
      feedback = None
  return grade, feedback

In [53]:
lst = []
for i,r in df_new.iterrows():
  lst.append(generate_mega_prompts(r.to_frame().T,0).loc[0,'MegaPrompt'])

df_new['0_mp'] = lst
df_new.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema,0_mp
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\n-1 pts for using t...,"SELECT O.OrderID, O.CustomerID\n , SUM(D.Qu...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...


In [55]:
from tqdm import tqdm
tqdm.pandas()

df_new['4_answer'] = df_new['0_mp'].progress_apply(GPT4_generation)
df_new['4_grade'], df_new['4_feedback'] = zip(*df_new['4_answer'].apply(parse_grade_feedback))
df_new.head()

100%|██████████| 3646/3646 [3:08:05<00:00,  3.10s/it]


,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,...,AnswerKey,isQuery,Database,full_score,feedback,schema,0_mp,4_answer,4_grade,4_feedback
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 9/10\n\nFeedback: Your answer is almost...,9,Your answer is almost perfect. You have correc...
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,...,"SELECT O.OrderID, O.CustomerID\n , SUM(D.Qu...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 8/10\n\nFeedback: Your SQL query is mos...,8,Your SQL query is mostly correct and it shows ...
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,...,"SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 9/10\n\nFeedback: Your SQL query is alm...,9,Your SQL query is almost perfect. You have cor...
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,...,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Excellent job! Your ...,10,Excellent job! Your query correctly retrieves ...
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,...,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query correctly retrieves ...


In [13]:
fs = gcsfs.GCSFileSystem(project="sql_autograding")
with fs.open("gs://sql_autograding/4_completion.csv") as f:
    df_new = pd.read_csv(f)
df_new = df_new.drop(columns='Unnamed: 0')
df_new.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,...,AnswerKey,isQuery,Database,full_score,feedback,schema,0_mp,4_answer,4_grade,4_feedback
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 9/10\n\nFeedback: Your answer is almost...,9,Your answer is almost perfect. You have correc...
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,...,"SELECT O.OrderID, O.CustomerID\n , SUM(D.Qu...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 8/10\n\nFeedback: Your SQL query is mos...,8,Your SQL query is mostly correct and it shows ...
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,...,"SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 9/10\n\nFeedback: Your SQL query is alm...,9,Your SQL query is almost perfect. You have cor...
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,...,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Excellent job! Your ...,10,Excellent job! Your query correctly retrieves ...
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,...,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query correctly retrieves ...


In [15]:
from tqdm import tqdm
tqdm.pandas()

from tenacity import(retry, stop_after_attempt, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT3_generation_with_backoff(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
df_new['3_answer'] = df_new['0_mp'].progress_apply(GPT3_generation_with_backoff)

In [18]:
df_new['3_grade'], df_new['3_feedback'] = zip(*df_new['3_answer'].apply(parse_grade_feedback))
df_new.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,...,full_score,feedback,schema,0_mp,4_answer,4_grade,4_feedback,3_answer,3_grade,3_feedback
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,...,10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 9/10\n\nFeedback: Your answer is almost...,9,Your answer is almost perfect. You have correc...,Grade: 7/10\n\nFeedback: \n- The student's que...,7,\n- The student's query is mostly correct and ...
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,...,10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 8/10\n\nFeedback: Your SQL query is mos...,8,Your SQL query is mostly correct and it shows ...,Grade: 8/10\n\nFeedback: \n- The student corre...,8,\n- The student correctly joins the Orders and...
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,...,7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 9/10\n\nFeedback: Your SQL query is alm...,9,Your SQL query is almost perfect. You have cor...,Grade: 7/10\n\nFeedback: \n- The query provide...,7,\n- The query provided by the student is almos...
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,...,10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Excellent job! Your ...,10,Excellent job! Your query correctly retrieves ...,Grade: 8/10\n\nFeedback: \n- The query provide...,8,\n- The query provided by the student is corre...
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,...,7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query correctly retrieves ...,Grade: 8/10\n\nFeedback: \n- The query provide...,8,\n- The query provided by the student is corre...


In [19]:
df_new.to_csv('completion.csv')